In [2]:
import os
from pprint import pprint
from solver.darp import Darp
from instance import parser as instance_parser
from solution import parser as solution_parser
import plot.route as route_plot
import json
import logging

# Change logging level to DEBUG to see the construction of the model
format = "%(asctime)s | %(name)s | %(levelname)s | %(message)s"
logging.basicConfig(level=logging.INFO, format=format)
logger = logging.getLogger()

result_folder = "./results/darp_heuristic/"
os.makedirs(result_folder, exist_ok=True)

instance_folder = "./instance/data/darp_cordeau_2006/"
filename = "a2-16"

filepath = os.path.join(instance_folder, filename)
instance = instance_parser.parse_instance_from_filepath(
    filepath, instance_parser=instance_parser.PARSER_TYPE_CORDEAU
)

In [40]:
data = instance.get_data()
data["dist_matrix"][0][0]
data["L"]

{1: 30,
 2: 30,
 3: 30,
 4: 30,
 5: 30,
 6: 30,
 7: 30,
 8: 30,
 9: 30,
 10: 30,
 11: 30,
 12: 30,
 13: 30,
 14: 30,
 15: 30,
 16: 30}

In [4]:
customers = data["P"]
customers_tw = [(i, data["el"][i]) for i in customers]
pprint(customers_tw)

[(1, (0, 1440)),
 (2, (0, 1440)),
 (3, (0, 1440)),
 (4, (0, 1440)),
 (5, (0, 1440)),
 (6, (0, 1440)),
 (7, (0, 1440)),
 (8, (0, 1440)),
 (9, (276, 291)),
 (10, (32, 47)),
 (11, (115, 130)),
 (12, (14, 29)),
 (13, (198, 213)),
 (14, (160, 175)),
 (15, (180, 195)),
 (16, (366, 381))]


 #### Step 1
 
 Index customers in the order of their earliest pick-up times:

In [5]:
customers_earliest = sorted(customers_tw, key=lambda x:x[1][0])
pprint(customers_earliest)

[(1, (0, 1440)),
 (2, (0, 1440)),
 (3, (0, 1440)),
 (4, (0, 1440)),
 (5, (0, 1440)),
 (6, (0, 1440)),
 (7, (0, 1440)),
 (8, (0, 1440)),
 (12, (14, 29)),
 (10, (32, 47)),
 (11, (115, 130)),
 (14, (160, 175)),
 (15, (180, 195)),
 (13, (198, 213)),
 (9, (276, 291)),
 (16, (366, 381))]


In [45]:
from collections import OrderedDict
def get_cost(k, route):
    capacity = 0
    previous = route[0]
    arrival_previous = 0
    service_previous = data["d"][previous]
    departure_previous = arrival_previous + service_previous
    cost = 0
    waiting = 0
    result = OrderedDict()
    result[previous] = dict(
        w=0,
        b=arrival_previous,
        t=0,
        q=capacity)
    
    for current in route[1:]:
        capacity+= data["q"][current]
        
        if capacity > data["Q"][k]:
            return -1, -1
        
        earliest_current = data["el"][current][0]
        latest_current = data["el"][current][1]
        
        travel_time = data["dist_matrix"][previous][current]
        arrival_current = departure_previous + travel_time
        
        print(f"{previous:>2} -> {travel_time:4.1f} -> {current:>2} ({earliest_current:>5.1f} - {arrival_current:>5.1f} - {latest_current:>6.1f} - cost = {cost:>7.2f}  - waiting = {waiting:>7.2f}")
        
        # Update previous
        if arrival_current <= latest_current:
            cost += travel_time
            waiting_at_current = max(earliest_current - arrival_current, 0)
            waiting += waiting_at_current
            arrival_current = max(departure_previous + travel_time, earliest_current)
            
            ride_delay = 0
            if current in data["D"]:
                i_delivery_node = data["D"].index(current)
                pickup_node = data["P"][i_delivery_node]
                
                departure_at_pickup = result[pickup_node]["b"]
                ride_delay = arrival_current - departure_at_pickup
                print(pickup_node, current, ride_delay, data["dist_matrix"][pickup_node][current])
                
                if ride_delay > data["L"][pickup_node]:
                    return -1, -1
                
            
            result[current] = dict(
                w=waiting_at_current,
                b=arrival_current,
                t=ride_delay,
                q=capacity)
        
            previous = current
            arrival_previous = arrival_current
            service_previous = data["d"][current]
            departure_previous = arrival_current + service_previous
        else:
            return -1, -1
    
    print(f"{previous:>2} -> {travel_time:4.1f} -> {current:>2} ({earliest_current:>5.1f} - {arrival_current:>5.1f} - {latest_current:>6.1f} - cost = {cost:>7.2f}  - waiting = {waiting:>7.2f}")
    pprint(result)
    return cost, waiting

work_schedule = dict()
work_schedule[0] = [0, 12, 6, 28, 22, 4, 11, 27, 20, 3, 19, 13, 29, 9, 8, 25, 24, 2, 18, 1, 17, 0]
work_schedule[1] =  [0, 10, 5, 26, 21, 14, 30, 15, 31, 7, 16, 23, 32, 0]


total_cost = 0
total_slack = 0
for k, route in work_schedule.items():
    cost, waiting = get_cost(k, route)
    total_cost+=cost
    total_slack+=waiting
    print(k, cost, waiting)
print(total_cost, total_slack)




 0 -> 10.0 -> 12 ( 14.0 -  10.0 -   29.0 - cost =    0.00  - waiting =    0.00
12 ->  2.9 ->  6 (  0.0 -  19.9 - 1440.0 - cost =    9.96  - waiting =    4.04
 6 ->  7.3 -> 28 (  0.0 -  30.2 - 1440.0 - cost =   12.90  - waiting =    4.04
12 28 16.214388941482863 10.005974415318082
28 -> 12.5 -> 22 ( 49.0 -  45.7 -   64.0 - cost =   20.17  - waiting =    4.04
6 22 29.055597853553287 17.972538078969258
22 ->  3.5 ->  4 (  0.0 -  55.5 - 1440.0 - cost =   32.67  - waiting =    7.33
 4 -> 14.4 -> 11 (115.0 -  73.0 -  130.0 - cost =   36.21  - waiting =    7.33
11 ->  5.0 -> 27 (  0.0 - 123.0 - 1440.0 - cost =   50.65  - waiting =   49.35
11 27 8.025990549135557 5.025990549135563
27 ->  1.9 -> 20 (138.0 - 128.0 -  153.0 - cost =   55.68  - waiting =   49.35
4 20 82.46728065083002 14.395074504843661
0 -1 -1
 0 ->  2.6 -> 10 ( 32.0 -   2.6 -   47.0 - cost =    0.00  - waiting =    0.00
10 -> 10.6 ->  5 (  0.0 -  45.6 - 1440.0 - cost =    2.61  - waiting =   29.39
 5 -> 10.1 -> 26 (  0.0 -  58.7

#### Step 2
For each vehicle $j (j = 1, 2, . . . , n)$:
   1. Find all the feasible ways in which customer $i$ can be inserted into the work-schedule of vehicle $j$. If it is infeasible to assign customer $i$ to vehicle $j$, examine the next vehicle $j + 1$ (restart *Step 1*); otherwise:
   2. Find the insertion of customer i into the work-schedule of vehicle j that results in minimum additional cost (details in Section 5). Call this additional cost COST,.

In [10]:
from collections import defaultdict
import math
work_schedule_vehicle_dict = defaultdict(list)

n = len(data["P"])

temp_work_schedule_vehicle_dict = defaultdict(list)
, 'q': 1.0}), (32, {'w': 0.5080375048326005, 'b': 414.0, 't': 30.0, 'q': 0.0}), ('0*', {'w': 0, 'b': 480.0, 't': 0, 'q': 0.0})]})])
# 0 [0, 12, 6, 28, 22, 4, 11, 27, 20, 3, 19, 13, 29, 9, 8, 25, 24, 2, 18, 1, 17, '0*']
# 1 [0, 10, 5, 26, 21, 14, 30, 15, 31, 7, 16, 23, 32, '0*']
def get_cost(route):
    o = route[0]
    arrival_o = 0
    service_o = data["d"][o]
    departure_o = arrival_o + service_o
    cost = 0
    slack = 0
    for d in route[1:]:
        travel_time_od = data["dist_matrix"][o][d]
        earliest_d = data["el"][d][0]
        arrival_d = max(departure_o + service_o + travel_time_od, earliest_d)
        latest_d = data["el"][d][1]
        
        # Update o
        if arrival_d <= latest_d:
            cost += travel_time_od
            slack += arrival_d - earliest_d
            o = d
            arrival_o = arrival_d
            service_o = data["d"][d]
            departure_o = arrival_d + service_o
        
        else:
            return -1, -1
    
    return cost, slack

def get_min_insertion(route, pickup, dropoff):
    for i in range(len(route)):
        temp_route = route.copy()
        temp_route.insert(i, pickup)
        for j in range(i+1, len(route)):
            temp_route.insert(j, dropoff)
            cost = get_cost(temp_route)
for k in data["K"]:
    work_schedule = work_schedule_vehicle_dict[k].copy()
    
    for pickup_node in customers_earliest:
        i_pickup_node = data["N"].index(pickup_node)
        delivery_node = data["N"][n + i_pickup_node + 1]
        get_min_insertion(temp_work_schedule_vehicle_dict)
        data["dist_matrix"][0][0]
    print(k)

KeyError: 'N'

In [ ]:
# ”, EPTi (i = 1, . . . , N), i.e. according to the earliest time at which they are expected to be available for a pick-up. Section 4 shows how EPT, is computed.

# result["instance"] = filename

# solution_obj = solution_parser.parse_solution_dict(result)

s# ol_filepath = f"{os.path.join(result_folder, filename)}.json"


# with open(sol_filepath, "w") as outfile:
#     json.dump(result, outfile, indent=4)

# fig, ax = route_plot.plot_vehicle_routes(
#     instance,
#     solution_obj,
#     jointly=True,
#     figsize=(10, 10),
#     show_arrows=False,
#     show_node_labels=False,
# )

# fig_filepath = f"{os.path.join(result_folder, filename)}.pdf"
# fig.savefig(fig_filepath, bbox_inches="tight")

NameError: name 's' is not defined